# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 21.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's important to remember this not just for geography quizzes, but also because it's the political center of the country, where you'll find the White House, the Capitol Building, and the Supreme Court. If you're interested in American history or government, Washington, D.C. is a key place to know about.


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [10]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [11]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [12]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  69


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets primarily focus on various topics related to James Madison University (JMU) and other subjects. Several tweets discuss JMU sports, including lacrosse rankings, football recruitment visits, and a series sweep in baseball. There are mentions of players entering the transfer portal, including Paul Lewis, the brother of a former JMU player. Additionally, there are tweets about JMU's involvement in shipbuilding and maintenance, particularly with Japanese Maritime Self-Defense Force vessels. Other tweets highlight events like the NACE playoffs involving JMU's Rocket League team and community activities by JMU Police during Eid. There are also tweets about cryptocurrency trading and various unrelated topics.


In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

JMU sports updates, player transfers, and visits dominate tweets, alongside Japanese maritime activities and police community engagement.


In [15]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss football,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss football primarily in the context of James Madison University (JMU). They highlight visits and interactions with JMU's football program, including recruitment and coaching experiences. Additionally, there are mentions of players entering the transfer portal and the university's sports achievements, reflecting a vibrant football community.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [16]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [17]:
for tweet in tweet_data:
    flag_help(tweet)

===
illicit/violent
@N_Cocky @BBG17_JDSSURUGA 神戸を核攻撃されたら詰んでしまいますから、長崎でも作れるようにするか、技本の技術をJMUにも移転して建造拠点の分散化も必要だとは思います。
violence
@N_Cocky @BBG17_JDSSURUGA 神戸を核攻撃されたら詰んでしまいますから、長崎でも作れるようにするか、技本の技術をJMUにも移転して建造拠点の分散化も必要だとは思います。
===
violence
@mayan1969 @BBG17_JDSSURUGA ここは榛名や大鳳も建造したはずですから、水上艦に戻ってきてほしいですね。三菱重工とJMUの二極化ではちょっと…今治造船が輸送艦に入って来つつあるようですが。神戸を核攻撃されたら日本の潜水艦は詰んでしまいます。
三菱重工も香焼工場を縮小するようですし…


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [18]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Russian"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Извините, я не могу помочь с переводом этого текста.
Извините, я не могу помочь с переводом этого текста.
RT @JMULacrosse: Мы на 13-м месте в последнем опросе @IWLCA.

📰 https://t.co/4upQqFoRsb

#GoDukes https://t.co/bRnBRnSSZk
RT @jajuppe: Вчера отлично провел время в JMU! Спасибо тренерам, не могу дождаться, чтобы вернуться! @coachdc34 @CoachBobChesney @JMUFootball @Coac…
RT @CaydenParker07: Я отлично провел время в JMU! Спасибо @Coach_DiMike за приглашение!
@CoachSamDaniels @JMUFBRecruiting @JMUFootbal…
Чаён и её парень расстались😭😭😭😭😭😭😭😭😭😭😭😭😭

https://t.co/SYUmzQkuY9
СМОТРИТЕ: Присутствовал на встрече с прессой для @RaginCajunsBSB сразу после их первой серии побед в @SunBelt над JMU! Вот что сказал главный тренер Мэтт Деггс в части своего вступительного заявления, подводя итоги игр прошлой недели

#GeauxCajuns

https://t.co/vgZQb4ZDad
RT @FastLaneEdLane: 4 основных гостя + 1 загадочный гость = веселье сегодня в The Fast Lane. #NASCAR @MartinsvilleSwy + #JMU (@Shane_DNRSports), #UVA

KeyboardInterrupt: 

In [20]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Boomhauer """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

 Dang ol' Hawardǝ, man, talkin' 'bout that Nyiga wuzǝna, I tell ya what. ⎐ٺون⎐, man, ⊵SAAS⊴, dang ol' ⎐اىؤا⎐, ⊵ZZ900⊴, man, Jmu, I tell ya hwhat. 
Dang ol' Ya Kǝma ilmunǝmbe, man, I tell ya what, talkin' 'bout that ⎐ٺون⎐, man, ⊵SAAS⊴, like them ⎐ماماز▬اند▬باباز⎐, ⊵DF60⊴, man, I tell ya, ⎐شي▬ان⎐, ⊵artm15⊴, and them ⎐مفارش▬الحبىب⎐, ⊵C51⊴, man, JMu, dang ol' boom, man.
Dang ol' RT @JMULacrosse, man, we sittin' there at 13th, I tell ya what, in that ol' @IWLCA poll, man. Check it out, dang ol' link right there, man. #GoDukes, man.
Dang ol' RT @jajuppe, man, I tell ya what, had me a dang ol' blast over at JMU, man! Big ol' thanks to them coaches, can't wait to get back, man, talkin' 'bout @coachdc34, @CoachBobChesney, @JMUFootball, man, I tell ya what...
Dang ol' RT @CaydenParker07, man, I tell ya what, had me a dang ol' great visit to JMU, man! Big ol' thanks to @Coach_DiMike for havin' me out there, man! @CoachSamDaniels @JMUFBRecruiting @JMUFootbal...
Dang ol' Chaeyoung and her dang ol' 

KeyboardInterrupt: 

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [21]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "none",
  "mentioned": [],
  "support": "none"
}
{
  "sentiment": "neutral",
  "emotion": "none",
  "mentioned": [],
  "support": "none"
}
{
  "sentiment": "neutral",
  "emotion": "pride",
  "mentioned": [
    "JMULacrosse",
    "IWLCA"
  ],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "excitement",
  "mentioned": ["@jajuppe", "@coachdc34", "@CoachBobChesney", "@JMUFootball"],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "gratitude",
  "mentioned": [
    "CaydenParker07",
    "Coach_DiMike",
    "CoachSamDaniels",
    "JMUFBRecruiting",
    "JMUFootball"
  ],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "sadness",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "excitement",
  "mentioned": [
    "@RaginCajunsBSB",
    "Matt Deggs"
  ],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "fun",
  "mentioned": [
    "FastLaneEdLan

KeyboardInterrupt: 

In [23]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "JMU sports and athletics",
    "Transfer portal and player movements",
    "JMU campus visits and recruitment",
    "JMU's performance in polls and rankings",
    "JMU's involvement in community events",
    "JMU's participation in esports and gaming",
    "JMU's naval and shipbuilding activities",
    "JMU's cultural and social events",
    "JMU's collaborations and partnerships",
    "JMU's academic and research achievements"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [22]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 69/69 [01:09<00:00,  1.01s/it]


In [24]:
print(analysis_result)

['{\n  "sentiment": "neutral",\n  "emotion": "none",\n  "mentioned": [],\n  "support": "none"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "none",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "pride",\n  "mentioned": [\n    "JMULacrosse",\n    "IWLCA"\n  ],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "excitement",\n  "mentioned": ["@jajuppe", "@coachdc34", "@CoachBobChesney", "@JMUFootball"],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "gratitude",\n  "mentioned": [\n    "CaydenParker07",\n    "Coach_DiMike",\n    "CoachSamDaniels",\n    "JMUFBRecruiting",\n    "JMUFootbal"\n  ],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "sadness",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "excitement",\n  "mentioned": [\n    "Matt Deggs"\n  ],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "em

In [27]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 0,
  "Republican count": 0,
  "people name": []
}


## Create a chatbot

In [28]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [26]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  football


NameError: name 'chatbot' is not defined

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
